<a href="https://colab.research.google.com/github/KevinTheRainmaker/ML_DL_Basics/blob/master/HPO/Keras_tuner_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Keras Tuner

Keras Tuner는 TensorFlow 기반 모델에서의 HPO를 도와주는 라이브러리로, 모델 하이퍼파라미터와 알고리즘 파라미터 모두를 아우른다.

해당 튜토리얼에서는 Keras Tuner를 이용하여 이미지 분류 애플리케이션에서 HPO를 수행한다.

### Configuration

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython # to delete train result before next train

In [2]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 12.6 MB/s 


In [4]:
import keras_tuner as kt

In [9]:
# mount gdrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path = '/content/drive/MyDrive/'

Mounted at /content/drive


### Download Datasets

In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### Define Model

HPO를 위해서는 모델 아키텍처뿐만 아니라 하이퍼파라미터를 조정할 수 있도록 설정해야한다. 이렇게 설정된 모델을 hyper model이라고 한다.

하이퍼 모델을 만드는 방법은 크게 두 가지로, 모델 빌더 함수를 사용하는 방법과 Keras Tuner API의 HyperModel 클래스를 사용하는 것이다.

여기서는 모델 아키텍처를 직접 정의해주기 위해 전자의 방식을 사용하도록 하겠다.

In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

### Instantiate the tuner and perform hypertuning

Keras Tuner는 하이퍼튜닝을 위한 튜너로 RandomSearch, HyperBand, Bayesian Optimization, Sklearn 총 4가지 튜너를 제공한다. 여기서는 HyperBand를 사용하도록 하겠다.

In [10]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory= path,
                     project_name='intro_to_kt')

In [11]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [12]:
# 하이퍼파라미터 탐색 실행
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 13s]
val_accuracy: 0.8773000240325928

Best val_accuracy So Far: 0.8878999948501587
Total elapsed time: 00h 14m 04s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 288 and the optimal learning rate for the optimizer
is 0.001.



위 과정에 의해 얻어진 최적 하이퍼파라미터 best_hps로 모델 재훈련

In [13]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4776 - accuracy: 0.8292 - val_loss: 0.4136 - val_accuracy: 0.8514
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3608 - accuracy: 0.8684 - val_loss: 0.3998 - val_accuracy: 0.8554
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3259 - accuracy: 0.8800 - val_loss: 0.3525 - val_accuracy: 0.8714
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2998 - accuracy: 0.8893 - val_loss: 0.3533 - val_accuracy: 0.8698
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2828 - accuracy: 0.8957 - val_loss: 0.3377 - val_accuracy: 0.8781
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2682 - accuracy: 0.9010 - val_loss: 0.3371 - val_accuracy: 0.8792
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2547 - accuracy: 0.9049 - val_loss: 0.3330 - val_accuracy:

Best val_accuracy: 0.8879

Test_accuracy: 0.8849

tuner에 의해 만들어진 intro_to_kt에는 HPO 중 수행되는 모든 모델 탐색의 상세 로그 및 체크포인트가 저장되어 있다.

하이퍼파라미터 탐색을 재실행할 경우 해당 폴더 내 로그를 확인하고 검색을 재개하며, 만약 이전의 로그를 무시하고 다시 시작하고 싶다면 튜너 instantiate 단계에서 overwrite = True 인수를 추가하면 된다.

참고자료:

https://www.tensorflow.org/tutorials/keras/keras_tuner